# <center> Générateurs de nombres aléatoires <br> TP2 - Registres à décalage à rétroaction linéaire (LFSR)</center>
<center> 2023/2024 - L. Naert/A. Ridard </center>

In [117]:
import datetime as dt
import numpy as np

## Le registre à décalage à rétroaction linéaire

### Exercice 1 : LFSR en tant que GNA

Considérons le LFSR dont le polynôme de rétroaction est défini par :
$$ f(X) = 1 + X^3 + X^5 + X^8 $$

Notons $\big(b_n\big)_{n\in\mathbb N}$ la suite générée par ce LFSR c'est à dire la suite récurrente binaire définie par :
$$ \left\{\begin{array}{l}
	b_0, b_1, \dots, b_7 \in \{0, 1\}\\ 
	\forall n\geq 8,\ b_{n} = b_{n-1}  \oplus b_{n-4} \oplus b_{n-6}
	\end{array}\right.$$
    
On peut représenter ce LFSR de la manière suivante :

<img src="LFSR_TP2.png" width="400">
    
Remarquons qu'il s'agit, en fait, du GMR d'ordre 8 défini par :

- $S = \{0, 1\}^8$
- $f(\textbf{s}) = f\Big(s^{(1)},\ \dots\ ,\ s^{(8)}\Big) = \Big(s^{(2)},\ \dots\ ,\ s^{(8)},\ s^{(3)} + \ s^{(5)}\  + s^{(8)} \mod 2\Big)$

Ou encore du générateur digital défini par :

- $S = \{0, 1\}^8$
- $f(\textbf{s}) = A\textbf{s} \mod 2$ où $A$ désigne la matrice carrée 
$\left(\begin{array}{cccccccc}
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\ 
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
0 & 0 & 1 & 0 & 1 & 0 & 0 & 1 \\
\end{array}\right)$

La fonction `generation_reg_graine(taille)` génère un registre de taille `taille` basée sur l'heure

In [118]:
def generation_reg_graine(taille):
    """
    Génération d'un registre de taille "taille" basée sur l'heure
    """ 

    ### Transformation de la date en une chaine de caractères
    date = str(dt.datetime.now())
    #print(date)
    ### Transformation de la fin de la chaine en un entier codable sur taille bits
    init_entier = int(date[-6:]) % 2**taille # j'ai choisi de prendre les 6 derniers caractères 
    #print(init_entier)
    ### Représentation de l'entier sur un octet
    init_bin = bin(init_entier)[2:] # on retire le 0b qui permet de préciser qu'il s'agit d'un nombre binaire
    while len(init_bin) < taille : 
        init_bin = '0' + init_bin # on rajoute des 0 pour que le nombre produit soit composé de taille bits. (padding)
    #print(init_bin)
    ### Transformation de la chaine des bits en une liste
    init_reg = [int(x) for x in init_bin]
    return init_reg

print(generation_reg_graine(8))

[1, 1, 0, 1, 1, 0, 1, 0]


> **Question 1.1 (Décalage d'un LFSR) :** Ecrire une fonction `etatSuivant(etat,iCoeff)` qui prend une liste binaire correspondant à l'état interne du registre ainsi qu'une liste des coefficients de rétroaction non nuls (i.e. indices des cases sur lesquelles faire le xor) et renvoie l'état suivant du registre et le bit de sortie.

ATTENTION : L'implémentation en Python impacte la numérotation : ici les coefficients de rétroaction seront indicés de $0$ à $l-1$ au lieu de $1$ à $l$ dans le cours.

In [119]:
def etatSuivant(etat,iCoeff):
    bitSortie=etat[0]
    last = 0
    etat=etat.copy()
    for i in iCoeff:
        last = last ^ etat[i]
    for i in range(len(etat)-1):
        etat[i]=etat[i+1]
    etat[len(etat)-1]=last
    return etat, bitSortie
    
try:
    assert etatSuivant([1,0,0,1],[0,2,3]) == ([0,0,1,0],1)  #Exemple du cours slide 14
    assert etatSuivant(etatSuivant([1,0,0,1],[0,2,3])[0],[0,2,3]) == ([0,1,0,1],0) #Exemple du cours slide 14
    print("etatSuivant : OK")
except:
    print("etatSuivant : ERREUR")

etatSuivant : OK


> **Question 1.2 (Période) :**
> 1) Ecrire une fonction `findPeriod(g,iCoeff)` qui renvoie la période ainsi que la liste des différents registres avant la première répétition. `graine` représente la graine du LFSR.
> 2) Tester cette fonction en utilisant la fonction de génération aléatoire d'une graine. Afficher les différents registres, indiquer la période et la période maximale possible.

In [120]:
def findPeriod(graine,iCoeff) :
    listEtat = [graine]
    periode = 0
    while True :
        etat, bitSortie = etatSuivant(listEtat[-1],iCoeff)
        if etat in listEtat :
            periode = len(listEtat) - listEtat.index(etat)
            break
        listEtat.append(etat)
    return listEtat, periode

try:
    lsfr_vect, p1= findPeriod([1,0,0,1],[0,2,3]) # LFSR du cours : périodique
    assert p1 == 7
    lsfr_vect, p2= findPeriod([1,0,1,0,1,1,0,0],[2,4,7]) # LFSR du TP : ultimement périodique 
    assert p2 == 7
    lsfr_vect, p3= findPeriod([1,0,1,0,1,1,0,0],[1,3,5])
    assert p3 == 63
    print("findPeriod : OK")
except:
    print("findPeriod : ERREUR")

findPeriod : OK


In [121]:
#Todo 2.
taille =8
graine=generation_reg_graine(taille)
iCoefs = [2,4,7]
listEtats, periode = findPeriod(graine,iCoefs)
print("periode = ",periode)
print("periode max = ",2**taille-1)
print("Liestes des etats avant repetion : ",listEtats)

periode =  7
periode max =  255
Liestes des etats avant repetion :  [[1, 0, 1, 1, 0, 0, 0, 1], [0, 1, 1, 0, 0, 0, 1, 0], [1, 1, 0, 0, 0, 1, 0, 1], [1, 0, 0, 0, 1, 0, 1, 1], [0, 0, 0, 1, 0, 1, 1, 0], [0, 0, 1, 0, 1, 1, 0, 0], [0, 1, 0, 1, 1, 0, 0, 0]]


A partir d'un état $\Big(s^{(1)},\ \dots\ ,s^{(8)}\Big)$ du registre, on peut générer un réel entre 0 et 1 (avec une précision de $2^{-8}$) défini par :
$$u = \displaystyle\sum_{i=1}^8 s^{(i)}2^{-i} \quad$$

Il s'agit, en fait, de la formule $g(\textbf{s}) = \displaystyle\sum_{i=1}^w \Big((B\textbf{s})_i \mod 2\Big)2^{-i}$ avec $B = I_8$.

En prenant $B = \left(\begin{array}{cccccccc}
		1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\ 
		0 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\
		0 & 0 & 1 & 0 & 0 & 0 & 0 & 0\\
		0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\
		\end{array}\right)$, on obtient toujours un réel entre 0 et 1 mais seulement avec une précision de $2^{-4}$ :
        
$$u = \displaystyle\sum_{i=1}^4 s^{(i)}2^{-i}$$

Plus généralement, on peut générer un réel entre 0 et 1 avec une précision de $2^{-w}$ grâce à la formule suivante ($w \le L$ avec $L$ la taille du registre): 

$$u = \displaystyle\sum_{i=1}^w s^{(i)}2^{-i} \quad (*)$$

> **Question 1.3 (Génération de réels) :**
1.  Programmer une fonction **regToReel**$(reg, w)$ qui transforme un registre $reg$ en un réel de précision de $2^{-w}$ suivant la formule (*). Si $w$ prend une valeur incorrect, la précision doit être mis à la valeur maximale possible.
2. Générer les réels (avec une précision de $2^{-8}$) jusqu'à la première répétition avec le LFSR présenté en début de TP et en initialisant les 8 bits avec l'horloge de l'ordinateur

In [122]:
def regToReel(reg,w) :
    reel = 0
    if len(reg) < w :
        w= len(reg)
    for i in range(w) :
        reel += reg[i]*2**(-i-1)
    return reel


try:
    assert regToReel([1,0,0,1],4) == 0.5625
    assert regToReel([1,0,0,1],3) == 0.5
    assert regToReel([1,0,1,1,0,1],18) == 0.703125
    print("regToReel : OK")
except:
    print("regToReel : ERREUR")


regToReel : OK


In [123]:
# Initialisation du registre et génération des réels jusqu'à l'obtention de la première répétition
for i in listEtats :
    print(regToReel(i,8))


0.69140625
0.3828125
0.76953125
0.54296875
0.0859375
0.171875
0.34375


### Exercice 2 : Application en cryptographie avec le masque jetable

Voici deux fonctions qui vous seront (certainement) utiles dans la suite du TP :

- `stringToBinary` convertit une chaine de caractère en une suite binaire.
- `binaryToString` permet de changer une suite binaire en chaine de caractère.

In [124]:
def stringToBinary(msg):
    msg_bin = ""
    for i in bytearray(msg, encoding ='ascii') :
        msg_bin = msg_bin + format(i, '08b')
    return msg_bin

def binaryToString(binary):
    msg = ""
    for i in range(0, len(binary), 8):
        byte_int = int(binary[i:i+8], 2)
        byte_char = chr(byte_int)
        msg = msg + byte_char
        
    return msg

        
print("En binaire :", stringToBinary("message en clair"))
print("En ascii :",binaryToString(stringToBinary("message en clair")))



En binaire : 01101101011001010111001101110011011000010110011101100101001000000110010101101110001000000110001101101100011000010110100101110010
En ascii : message en clair


Le masque jetable, aussi appelé "chiffrement de Vernam", repose sur le principe du "ou exclusif" (_xor_, noté $\oplus$, opérateur ^ en python) bit à bit entre le message binaire à chiffrer et la clef de chiffrement (de même longueur). 

Voici la table de vérité du "ou exclusif" :
$$ 0 \oplus 0 = 0 $$ 
$$ 1 \oplus 1 = 0 $$
$$ 1 \oplus 0 = 1 $$
$$ 0 \oplus 1 = 1 $$

Ainsi, étant donné une clef _k_ de longueur _n_ (donc $k \in (\mathbb{Z}/2\mathbb{Z})^n$)

\begin{align*}
  E_k \colon (\mathbb{Z}/2\mathbb{Z})^n &\to (\mathbb{Z}/2\mathbb{Z})^n\\
  m &\mapsto c = m \oplus k
\end{align*}

Par exemple, avec $m = 1100 1100$ et $k = 1010 1011$, on aura $c = 0110 0111$ (Vérifiez par vous même !)

Le masque jetable garantit la sécurité des messages à condition qu'une clef ne serve qu'au chiffrement d'un seul message (d'où le "jetable" du nom) sinon, la cryptanalyse devient possible.


> __Question 2.1 (masque jetable/Chiffrement de Vernam)__ : Définir une fonction `chiffrementVernam(msgBinaire, clef)` qui étant donné un message en clair binaire `msgBinaire`, et une suite binaire `clef` de même longueur que le message retourne le message chiffré correspondant.

In [125]:
def chiffrementVernam(msgBinaire, clef):
    chiffre = ""
    for i in range(len(msgBinaire)):
        chiffre+=str(int(msgBinaire[i]) ^ int(clef[i]))
    return chiffre

try:
    assert chiffrementVernam(stringToBinary("vernam"),"110011001100110011001100110011001100110011001100") == "101110101010100110111110101000101010110110100001"
    print("chiffrementVernam : OK")
except:
    print("chiffrementVernam : ERREUR")

chiffrementVernam : OK


Le déchiffrement s'opère en executant la même opération : $m = c \oplus k$
> __Question 2.2 (déchiffrement)__ : Quel clair (en ascii) représente le chiffré "1010001010101101101010011011111010111000" codé avec la clef "1100110011001100110011001100110011001100" ? Ecrire le bout de code permettant de le déchiffrer.



In [126]:
coded="1010001010101101101010011011111010111000"
key="1100110011001100110011001100110011001100"

string = ""
for i in range(len(coded)):
    string+=str(int(coded[i]) ^ int(key[i]))
print(binaryToString(string))

naert


> __Question 2.3 (suite chiffrante)__ :
> - Ecrire une fonction `suite_LSFR(graine,coeff,n)` qui prend en entrée une liste binaire correspondant à la graine du registre, une liste des indices des coefficients de rétroaction non nuls et la longueur souhaitée de la suite chiffrante et qui renvoie la suite chiffrante binaire __sous forme d'une chaine de caractère__.
> - Ecrire une fonction `convertToList(suiteCaractere)` qui prend une chaine de caractère composée de 0 et de 1 et la convertit en liste d'entier.

In [127]:
def suite_LFSR(graine,iCoeff,n):
    suite=""
    for i in range(n):
        graine, bitSortie = etatSuivant(graine,iCoeff)
        suite += str(bitSortie)
    return suite

def convertToList(suiteCaractere):
    chars=list(suiteCaractere)
    liste=[]
    for i in chars:
        liste.append(int(i))
    return liste
    
try:
    assert suite_LFSR([1,0,0,1],[0,2,3],14) == "10010111001011"
    assert convertToList("10010111001011") == [1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1]
    print("suite_LFSR et convertToList : OK")
except:
    print("suite_LFSR et convertToList : ERREUR")


suite_LFSR et convertToList : OK


> __Question 2.4 (Chiffrement par LFSR)__ : Ecrire une fonction `chiffrementLFSR(msgAscii, graine,coeff)` qui déroule l'ensemble du processus de chiffrement par masque jetable généré par LSFR et retourne la version binaire du message chiffré.

In [128]:
def chiffrementLFSR(msgAscii, graine,iCoeff):
    bin=stringToBinary(msgAscii)
    key = suite_LFSR(graine,iCoeff,len(bin))
    msgchiffre = chiffrementVernam(bin,key)
    return msgchiffre

try:
    assert chiffrementLFSR("naert",[1,0,0,1],[0,2,3]) == "1111100101001111001110011100101100000110"
    print("chiffrementLFSR : OK")
except:
    print("chiffrementLFSR : ERREUR")


chiffrementLFSR : OK


### Exercice 3 : Attaque sur un LFSR et reconstruction du polynôme de rétroaction minimal

Si l'attaquant dispose des 16 premiers bits $b_0,\ \dots\ ,\ b_{15}$, il peut reconstruire le polynôme de rétroaction
$f(X) = 1 + c_1X + \dots + c_8X^8$.

Il suffit, en effet, de résoudre l'équation matricielle :
$$ \left(\begin{array}{cccc}
		b_0 & b_1 & \dots & b_7 \\
        b_1 & b_2 & \dots & b_8 \\
        \vdots & & & \vdots \\
        b_7 & b_8 & \dots & b_{14} \\
		\end{array}\right) . 
        \left(\begin{array}{c}
		c_1 \\ 
		c_2 \\
        \vdots \\
        c_8 \\
		\end{array}\right) =
        \left(\begin{array}{c}
		b_8 \\ 
		b_9 \\
		\vdots \\
        b_{15} \\
		\end{array}\right)
        $$
        
Pour cela, on peut utiliser la méthode du pivot de Gauss pour résoudre le système d'équations linéaires associé d'inconnues $c_1,\ \dots\ ,\ c_8$.

On résume ce système avec la "matrice étendue" $\left(\begin{array}{cccc | c}
		b_0 & b_1 & \dots & b_7 & b_8 \\
        b_1 & b_2 & \dots & b_8 & b_9 \\
        \vdots & & &  & \vdots \\
        b_7 & b_8 & \dots & b_{14} & b_{15} \\
		\end{array}\right)$.
        
On applique alors l'algorithme suivant :


- Pour $i$ variant de 0 à 7 :

    - (A) : si la ligne $i$ ne contient pas 1 en position $i$, on cherche une ligne $k$ contenant 1 en position $i$ et on remplace la ligne $i$ par son XOR avec la ligne $k$. $k$ > $i$
    
    - (B) : on remplace chaque ligne qui contient 1 en colonne $i$, sauf la $i$-ième, par son XOR avec la ligne $i$
$$ $$
- Si la boucle s'est bien terminée, on obtient l'unique solution dans la dernière colonne (attention à l'ordre)

- Si le premier point dans la boucle échoue, c'est que la matrice n'est pas inversible car la complexité linéaire est en fait strictement inférieure à 8...

Dans ce cas, pour déterminer la complexité linéaire et le polynôme de rétroaction minimal, on exécute l'algorithme précédent pour toutes les complexités linéaires $l$ possibles $1,\ \dots,\ 7$ en partant de $l=1$, jusqu'à pouvoir reconstituer tous les termes connus de la suite.<br>
Chaque résolution de système, à partir des $2l$ premiers bits $b_0,\ \dots,\ b_{2l-1}$ fournit un polynôme de rétroaction candidat que l'on teste avec les termes suivants de la suite $b_{2l},\ \dots,\ b_{15}$ pour savoir s'il convient.

> **Question 3.1 :** Coder une fonction `suiteToSys(suiteChiff)` qui transforme une suite chiffrante sous forme de liste d'entier de taille 2l en système/matrice de taille $l \times (l+1)$ (à résoudre par le pivot de Gauss)

In [129]:
# On transforme une suite chiffrante en système (à résoudre par le pivot de Gauss)

def suiteToSys(suiteChiff) :
    """
    Cette fonction retourne le système, sous forme de matrice étendue l x l+1, 
    provenant de la suite chiffrante de longueur 2l
    """
    l=len(suiteChiff)//2
    sys = np.zeros((l,l+1),dtype=int)
    for i in range(l):
        sys[i][l]=suiteChiff[i]
        for j in range(l+1):
            sys[i][j]=suiteChiff[i+j]
        
        
    #todo
    return sys

init =  [0, 0, 1, 1, 1, 0, 1, 1]
suiteChiff = convertToList(suite_LFSR(init, [2, 4, 7], 16))
print('La suite chiffrante générée à partir de la graine', init, 'est :')
print(suiteChiff, '\n')

tabSys = suiteToSys(suiteChiff)
print('Le système à résoudre est alors :')
print(tabSys,"\n")

try : 
    tabSys = suiteToSys(suiteChiff)
    assert np.all(tabSys == np.array([[0, 0, 1, 1, 1, 0, 1, 1, 1], 
                            [0, 1, 1, 1, 0, 1, 1, 1, 0], 
                            [1, 1, 1, 0, 1, 1, 1, 0, 0],
                            [1, 1, 0, 1, 1, 1, 0, 0, 1],
                            [1, 0, 1, 1, 1, 0, 0, 1, 1],
                            [0, 1, 1, 1, 0, 0, 1, 1, 0],
                            [1, 1, 1, 0, 0, 1, 1, 0, 1],
                            [1, 1, 0, 0, 1, 1, 0, 1, 0]]))
    print("suiteToSys : OK")
except:
    print("suiteToSys : ERREUR")

La suite chiffrante générée à partir de la graine [0, 0, 1, 1, 1, 0, 1, 1] est :
[0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0] 

Le système à résoudre est alors :
[[0 0 1 1 1 0 1 1 1]
 [0 1 1 1 0 1 1 1 0]
 [1 1 1 0 1 1 1 0 0]
 [1 1 0 1 1 1 0 0 1]
 [1 0 1 1 1 0 0 1 1]
 [0 1 1 1 0 0 1 1 0]
 [1 1 1 0 0 1 1 0 1]
 [1 1 0 0 1 1 0 1 0]] 

suiteToSys : OK


> **Question 3.2 :**
> Ecrire une fonction `pivotGauss(matrice)` qui prend une matrice de taille l lignes, l+1 colonnes, qui execute l'algorithme du pivot de gauss et qui retourne la liste des positions des 1 lorsque la solution est unique, et -1 sinon (matrice non inversible)



In [130]:
# On commence par le pivot de Gauss
def xorArr(arr1,arr2):
    res=[]
    for i in range(len(arr1)):
        res.append(arr1[i]^arr2[i])
    return res


def pivotGauss(matrice) :
    """
    Cette fonction retourne la liste des positions des 1 lorsque la solution est unique, 
    et -1 sinon (matrice non inversible)
    """
    systeme=matrice
    res=[]
    if np.linalg.det(matrice[:,:-1]) == 0:
        return -1,systeme

    # Step (A)
    for i in range(len(systeme)):
        if systeme[i][i] != 1:
            for k in range(i+1, len(systeme)):
                if systeme[k][i] == 1:
                    systeme[i] = xorArr(systeme[i], systeme[k])
                    break

        # Step (B)
        for j in range(len(systeme)):
            if systeme[j][i] == 1 and j != i:
                systeme[j] = xorArr(systeme[i], systeme[j])



    for i in range(len(systeme)):
        if systeme[i][len(systeme)] == 1:
            res.append(i)
    return(res,systeme)

try: 
    #Un exemple de résolution avec une matrice inversible, mais qui ne provient pas d'une suite chiffrante !
    tabSys = np.array([[1, 1, 1, 1, 1, 0], [0, 0, 1, 1, 0, 1], [1, 1, 0, 1, 0, 0], [0, 0, 1, 1, 1, 0], 
                    [0, 1, 1, 1, 1, 0]])
    sol, sysResolu = pivotGauss(tabSys)
    print(sol,"\n")
    print(sysResolu,"\n")
    print(np.array([[1, 0, 0, 0, 0, 0],[0, 1, 0, 0, 0, 0],[0, 0, 1, 0, 0, 1],[0, 0, 0, 1, 0, 0],[0, 0, 0, 0, 1, 1]]))
    assert sol == [2, 4]
    assert np.all(sysResolu == np.array([[1, 0, 0, 0, 0, 0],[0, 1, 0, 0, 0, 0],[0, 0, 1, 0, 0, 1],[0, 0, 0, 1, 0, 0],[0, 0, 0, 0, 1, 1]]))
    
    print("pivotGauss : OK")
except:
    print("pivotGauss : ERREUR",)
    

[2, 4] 

[[1 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 0 1]
 [0 0 0 1 0 0]
 [0 0 0 0 1 1]] 

[[1 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 0 1]
 [0 0 0 1 0 0]
 [0 0 0 0 1 1]]
pivotGauss : OK


> **Question 3.3 :**
> Ecrire une fonction `indicesToPolynome(iCoeff)` qui a partir de la liste des coefficients `iCoeff` renvoie le polynome de réatroaction sous forme d'une chaine de caractère. 

In [131]:
def indicesToPolynome(iCoeff):
    #todo
    polynome = "f(X) = 1 + "
    for i in iCoeff:
        polynome += "X**"+str(i+1)
        if i != iCoeff[-1]:
            polynome += " + "
    return polynome


try : 
    assert indicesToPolynome([2,4,7]) == "f(X) = 1 + X**3 + X**5 + X**8"
    print("indicesToPolynome : OK")
except:
    print("indicesToPolynome : ERREUR")

indicesToPolynome : OK


> **Question 3.4 :**
1. Rassembler les éléments précédents dans une fonction `attaque_LFSR(suiteChiff)` qui retourne la complexité linéaire (inférieure ou égale à $l$) et les indices des coefficients de rétroaction à partir de la liste $suiteChiff$ des $2l$ premiers bits $b_0,\ \dots,\ b_{2l-1}$
2. Tester la fonction avec notre LFSR : faire en sorte d'afficher la complèxité linéaire et le polynome de rétroaction minimal.

In [135]:
def attaque_LFSR(suiteChiff, verbose = False):
    complexite = 2
    coeff = []
    while complexite < len(suiteChiff)//2:
        sys = suiteToSys(suiteChiff[:2*complexite])
        sol, sysResolu = pivotGauss(sys)
        if sol != -1:
            if convertToList(suite_LFSR(suiteChiff[:complexite], sol, len(suiteChiff))) == suiteChiff:
                coeff = sol
                break
        complexite += 1
    
    return complexite, coeff


try : 
    init = [0, 1, 1, 1, 1, 0, 0, 0]
    comp, coeff = attaque_LFSR(suiteChiff)
    print(comp, coeff)
    assert comp == 6
    assert coeff == [1,2]
    init = [0, 0, 0, 1, 1, 0, 1, 1] 
    suiteChiff = convertToList(suite_LFSR(init, [2, 4, 7], 16))
    comp, coeff = attaque_LFSR(suiteChiff)
    assert comp == 4
    assert coeff == [2, 3] 
    print("attaque_LFSR : OK")
except:
    print("attaque_LFSR : ERREUR")



6 [1, 2]
attaque_LFSR : OK


In [136]:
init = [0, 1, 1, 1, 1, 0, 0, 0]
init2 = [0, 0, 1, 1, 1, 0, 1, 1]
init3 = [1, 1, 0, 1, 1, 0, 0, 1]
init4 = [0, 0, 0, 1, 1, 0, 1, 1] 
suiteChiff = convertToList(suite_LFSR(init, [2, 4, 7], 16))
suiteChiff2 = convertToList(suite_LFSR(init2, [2, 4, 7], 16))
suiteChiff3 = convertToList(suite_LFSR(init3, [2, 4, 7], 16))
suiteChiff4 = convertToList(suite_LFSR(init4, [2, 4, 7], 16))
#Quels polynomes de rétroaction obtenez vous ?

print(attaque_LFSR(suiteChiff)[0], indicesToPolynome(attaque_LFSR(suiteChiff)[1]))
print(attaque_LFSR(suiteChiff2)[0], indicesToPolynome(attaque_LFSR(suiteChiff2)[1]))
print(attaque_LFSR(suiteChiff3)[0], indicesToPolynome(attaque_LFSR(suiteChiff3)[1]))
print(attaque_LFSR(suiteChiff4)[0], indicesToPolynome(attaque_LFSR(suiteChiff4)[1]))


6 f(X) = 1 + X**2 + X**3
8 f(X) = 1 + 
7 f(X) = 1 + X**3 + X**4
4 f(X) = 1 + X**3 + X**4


Notez que la complexité linéaire est liée à la suite chiffrante (et donc à la graine) et pas au LFSR ! 